<a href="https://colab.research.google.com/github/sshivaditya2019/CompAutoEnco/blob/main/SimpleAutoEncoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>